In [ ]:
import cv2

# Load the video file
video_path = "test2.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()  # Read a frame

    if not ret:
        break  # Break the loop if no frame is read (end of video)

    cv2.imshow("Video Player", frame)  # Display the frame

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2

# Load the video file
video_path = "test2.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

while True:
    ret, frame = cap.read()  # Read a frame

    if not ret:
        break  # Break the loop if no frame is read (end of video)

    # Get frame dimensions
    height, width, _ = frame.shape
    center_x, center_y = width // 2, height // 2  # Compute center

    # Draw a red dot at the center
    cv2.circle(frame, (center_x, center_y), radius=5, color=(0, 0, 255), thickness=-1)

    cv2.imshow("Video Player", frame)  # Display the frame

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the video file
#video_path = "test.mp4"  # Replace with your video file path

# Replace with your phone's IP address and port
url = "https://192.168.185.1:8080/video"

#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(url)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    if descriptors is not None and prev_descriptors is not None:
        # Match descriptors
        matches = bf.match(prev_descriptors, descriptors)

        # Sort matches by distance (lower is better)
        matches = sorted(matches, key=lambda x: x.distance)

        # Calculate average displacement of keypoints
        motion_x = []
        motion_y = []
        
        for m in matches:
            pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
            pt2 = keypoints[m.trainIdx].pt  # Current keypoint

            motion_x.append(pt2[0] - pt1[0])  # X movement
            motion_y.append(pt2[1] - pt1[1])  # Y movement

        if motion_x and motion_y:
            avg_x = np.mean(motion_x)  # Average movement in X
            avg_y = np.mean(motion_y)  # Average movement in Y
            total_movement = np.sqrt(avg_x**2 + avg_y**2)

            # Set threshold for significant movement (adjust as needed)
            if total_movement > 120:  
                movement_text = "Camera Moving"
                color = (0, 0, 255)  # Red
            else:
                movement_text = "Camera Static"
                color = (0, 255, 0)  # Green
            
            # Display movement status
            cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow("Camera Movement Detection", frame)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import time

# Replace with your phone's IP address and port
url = "http://192.168.185.1:8080/video"  # Ensure HTTP, not HTTPS

# Function to handle reconnection
def reconnect_camera():
    while True:
        cap = cv2.VideoCapture(url)
        if cap.isOpened():
            print("Reconnected to the camera.")
            return cap
        print("Reconnecting to the camera...")
        cv2.waitKey(1000)  # Wait 1 sec before retrying

# Initialize video capture
cap = reconnect_camera()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Canvas for plotting dots
canvas_height, canvas_width = 300, 800  # Canvas size
canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # White background
dot_position = 50  # Starting X position for dots
dot_spacing = 5  # Closer spacing to make a line

# Timer to skip first 10 seconds
start_time = time.time()
skip_time = 10  # Seconds to skip movement detection

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Camera disconnected. Attempting to reconnect...")
        cap = reconnect_camera()
        continue  # Skip frame processing until reconnection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Skip movement detection for the first 10 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time < skip_time:
        cv2.putText(frame, "Initializing...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else:
        if descriptors is not None and prev_descriptors is not None:
            # Match descriptors
            matches = bf.match(prev_descriptors, descriptors)
            matches = sorted(matches, key=lambda x: x.distance)  # Sort by best match

            # Calculate average displacement of keypoints
            motion_x = []
            motion_y = []
            
            for m in matches:
                pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
                pt2 = keypoints[m.trainIdx].pt  # Current keypoint

                motion_x.append(pt2[0] - pt1[0])  # X movement
                motion_y.append(pt2[1] - pt1[1])  # Y movement

            if motion_x and motion_y:
                avg_x = np.mean(motion_x)  # Average movement in X
                avg_y = np.mean(motion_y)  # Average movement in Y
                total_movement = np.sqrt(avg_x**2 + avg_y**2)

                # Set threshold for significant movement
                if total_movement > 120:  # Keep the original threshold
                    movement_text = "Camera Moving"
                    color = (0, 0, 255)  # Red
                else:
                    movement_text = "Camera Static"
                    color = (0, 255, 0)  # Green

                    # Plot red dot in a straight line when the camera is static
                    if dot_position < canvas_width - 50:  # Ensure dots don't overflow
                        cv2.circle(canvas, (dot_position, canvas_height // 2), 5, (0, 0, 255), -1)
                        dot_position += dot_spacing  # Move position for next dot (closer spacing)
                    else:
                        dot_position = 50  # Reset dots to start when reaching the end

                # Display movement status
                cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the main video feed
    cv2.imshow("Camera Movement Detection", frame)

    # Show the dot plotting canvas
    cv2.imshow("Static Camera Plot", canvas)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import time

# Replace with your phone's IP address and port
url = "http://192.168.185.1:8080/video"  # Ensure HTTP, not HTTPS

# Function to handle reconnection
def reconnect_camera():
    while True:
        cap = cv2.VideoCapture(url)
        if cap.isOpened():
            print("Reconnected to the camera.")
            return cap
        print("Reconnecting to the camera...")
        cv2.waitKey(1000)  # Wait 1 sec before retrying

# Initialize video capture
cap = reconnect_camera()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Canvas for plotting dots
canvas_height, canvas_width = 300, 800  # Canvas size
canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # White background

dot_position_x, dot_position_y = 50, canvas_height // 2  # Initial dot position
dot_spacing = 10  # Dots appear only as frames progress
direction = 0  # 0 = Right, 1 = Down, 2 = Left, 3 = Up
frames_since_last_dot = 0  # Ensures dots appear as frames progress

# Timer to skip first 10 seconds
start_time = time.time()
skip_time = 10  # Seconds to skip movement detection

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Camera disconnected. Attempting to reconnect...")
        cap = reconnect_camera()
        continue  # Skip frame processing until reconnection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Skip movement detection for the first 10 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time < skip_time:
        cv2.putText(frame, "Initializing...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else:
        if descriptors is not None and prev_descriptors is not None:
            # Match descriptors
            matches = bf.match(prev_descriptors, descriptors)
            matches = sorted(matches, key=lambda x: x.distance)  # Sort by best match

            # Calculate average displacement of keypoints
            motion_x = []
            motion_y = []
            
            for m in matches:
                pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
                pt2 = keypoints[m.trainIdx].pt  # Current keypoint

                motion_x.append(pt2[0] - pt1[0])  # X movement
                motion_y.append(pt2[1] - pt1[1])  # Y movement

            if motion_x and motion_y:
                avg_x = np.mean(motion_x)  # Average movement in X
                avg_y = np.mean(motion_y)  # Average movement in Y
                total_movement = np.sqrt(avg_x**2 + avg_y**2)

                # Set threshold for significant movement
                if total_movement > 120:  
                    movement_text = "Camera Moving"
                    color = (0, 0, 255)  # Red

                    # Rotate direction on movement but keep dot at last position
                    direction = (direction + 1) % 4  

                else:
                    movement_text = "Camera Static"
                    color = (0, 255, 0)  # Green

                    # Add a dot only when enough frames have passed (not continuous)
                    frames_since_last_dot += 1
                    if frames_since_last_dot >= 5:  # Adjust for spacing
                        cv2.circle(canvas, (dot_position_x, dot_position_y), 5, (0, 0, 255), -1)

                        # Move position based on direction
                        if direction == 0:  # Right
                            dot_position_x += dot_spacing
                            if dot_position_x >= canvas_width - 50:
                                dot_position_x = canvas_width - 50
                        elif direction == 1:  # Down
                            dot_position_y += dot_spacing
                            if dot_position_y >= canvas_height - 50:
                                dot_position_y = canvas_height - 50
                        elif direction == 2:  # Left
                            dot_position_x -= dot_spacing
                            if dot_position_x <= 50:
                                dot_position_x = 50
                        elif direction == 3:  # Up
                            dot_position_y -= dot_spacing
                            if dot_position_y <= 50:
                                dot_position_y = 50

                        frames_since_last_dot = 0  # Reset counter after drawing a dot

                # Display movement status
                cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the main video feed
    cv2.imshow("Camera Movement Detection", frame)

    # Show the dot plotting canvas
    cv2.imshow("Static Camera Plot", canvas)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import time

# Replace with your phone's IP address and port
url = "http://192.168.185.1:8080/video"  # Ensure HTTP, not HTTPS

# Function to handle reconnection
def reconnect_camera():
    while True:
        cap = cv2.VideoCapture(url)
        if cap.isOpened():
            print("Reconnected to the camera.")
            return cap
        print("Reconnecting to the camera...")
        cv2.waitKey(1000)  # Wait 1 sec before retrying

# Initialize video capture
cap = reconnect_camera()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Canvas for plotting dots
canvas_height, canvas_width = 300, 800  # Canvas size
canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # White background

dot_position_x, dot_position_y = 50, canvas_height // 2  # Initial dot position
dot_spacing = 10  # Dots appear only as frames progress
direction = 0  # 0 = Right, 1 = Down, 2 = Left, 3 = Up
frames_since_last_dot = 0  # Ensures dots appear as frames progress

# Timer to skip first 10 seconds
start_time = time.time()
skip_time = 10  # Seconds to skip movement detection
last_rotation_time = 0  # Timer for 5s delay between rotations
rotation_delay = 5  # 5 seconds delay before next rotation

# Store drawn points to prevent intersections
drawn_points = set()

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Camera disconnected. Attempting to reconnect...")
        cap = reconnect_camera()
        continue  # Skip frame processing until reconnection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Skip movement detection for the first 10 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time < skip_time:
        cv2.putText(frame, "Initializing...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else:
        if descriptors is not None and prev_descriptors is not None:
            # Match descriptors
            matches = bf.match(prev_descriptors, descriptors)
            matches = sorted(matches, key=lambda x: x.distance)  # Sort by best match

            # Calculate average displacement of keypoints
            motion_x = []
            motion_y = []
            
            for m in matches:
                pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
                pt2 = keypoints[m.trainIdx].pt  # Current keypoint

                motion_x.append(pt2[0] - pt1[0])  # X movement
                motion_y.append(pt2[1] - pt1[1])  # Y movement

            if motion_x and motion_y:
                avg_x = np.mean(motion_x)  # Average movement in X
                avg_y = np.mean(motion_y)  # Average movement in Y
                total_movement = np.sqrt(avg_x**2 + avg_y**2)

                # Set threshold for significant movement
                if total_movement > 120:  
                    movement_text = "Camera Moving"
                    color = (0, 0, 255)  # Red

                    # Rotate direction only if delay period has passed
                    current_time = time.time()
                    if current_time - last_rotation_time >= rotation_delay:
                        direction = (direction + 1) % 4  # Rotate direction
                        last_rotation_time = current_time  # Update last rotation time

                else:
                    movement_text = "Camera Static"
                    color = (0, 255, 0)  # Green

                    # Add a dot only when enough frames have passed (not continuous)
                    frames_since_last_dot += 1
                    if frames_since_last_dot >= 5:  # Adjust for spacing
                        # Prevent intersections by checking if the position is already occupied
                        if (dot_position_x, dot_position_y) not in drawn_points:
                            cv2.circle(canvas, (dot_position_x, dot_position_y), 5, (0, 0, 255), -1)
                            drawn_points.add((dot_position_x, dot_position_y))  # Store the point

                            # Move position based on direction
                            if direction == 0:  # Right
                                dot_position_x += dot_spacing
                                if dot_position_x >= canvas_width - 50:
                                    dot_position_x = canvas_width - 50
                            elif direction == 1:  # Down
                                dot_position_y += dot_spacing
                                if dot_position_y >= canvas_height - 50:
                                    dot_position_y = canvas_height - 50
                            elif direction == 2:  # Left
                                dot_position_x -= dot_spacing
                                if dot_position_x <= 50:
                                    dot_position_x = 50
                            elif direction == 3:  # Up
                                dot_position_y -= dot_spacing
                                if dot_position_y <= 50:
                                    dot_position_y = 50

                        frames_since_last_dot = 0  # Reset counter after drawing a dot

                # Display movement status
                cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the main video feed
    cv2.imshow("Camera Movement Detection", frame)

    # Show the dot plotting canvas
    cv2.imshow("Static Camera Plot", canvas)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import time

# Replace with your phone's IP address and port
url = "http://192.168.185.1:8080/video"  # Ensure HTTP, not HTTPS

# Function to handle reconnection
def reconnect_camera():
    while True:
        cap = cv2.VideoCapture(url)
        if cap.isOpened():
            print("Reconnected to the camera.")
            return cap
        print("Reconnecting to the camera...")
        cv2.waitKey(1000)  # Wait 1 sec before retrying

# Initialize video capture
cap = reconnect_camera()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Get full-screen resolution
screen_width = 1920
screen_height = 1080

# Canvas for plotting dots (Full-screen size)
canvas = np.ones((screen_height, screen_width, 3), dtype=np.uint8) * 255  # White background

dot_position_x, dot_position_y = 50, 50  # Start at the **top-left corner**
dot_spacing = 20  # Increased spacing to slow down plotting
direction = 0  # 0 = Right, 1 = Down, 2 = Left, 3 = Up
frames_since_last_dot = 0  # Ensures dots appear as frames progress

# Timer to skip first 10 seconds
start_time = time.time()
skip_time = 10  # Seconds to skip movement detection
last_rotation_time = 0  # Timer for 5s delay between rotations
rotation_delay = 8  # 5 seconds delay before next rotation

# Store drawn points to prevent intersections
drawn_points = set()

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Camera disconnected. Attempting to reconnect...")
        cap = reconnect_camera()
        continue  # Skip frame processing until reconnection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Skip movement detection for the first 10 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time < skip_time:
        cv2.putText(frame, "Initializing...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else:
        if descriptors is not None and prev_descriptors is not None:
            # Match descriptors
            matches = bf.match(prev_descriptors, descriptors)
            matches = sorted(matches, key=lambda x: x.distance)  # Sort by best match

            # Calculate average displacement of keypoints
            motion_x = []
            motion_y = []
            
            for m in matches:
                pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
                pt2 = keypoints[m.trainIdx].pt  # Current keypoint

                motion_x.append(pt2[0] - pt1[0])  # X movement
                motion_y.append(pt2[1] - pt1[1])  # Y movement

            if motion_x and motion_y:
                avg_x = np.mean(motion_x)  # Average movement in X
                avg_y = np.mean(motion_y)  # Average movement in Y
                total_movement = np.sqrt(avg_x**2 + avg_y**2)

                # Set threshold for significant movement
                if total_movement > 200:  
                    movement_text = "Camera Moving"
                    color = (0, 0, 255)  # Red

                    # Rotate direction only if delay period has passed
                    current_time = time.time()
                    if current_time - last_rotation_time >= rotation_delay:
                        direction = (direction + 1) % 4  # Rotate direction
                        last_rotation_time = current_time  # Update last rotation time

                else:
                    movement_text = "Camera Static"
                    color = (0, 255, 0)  # Green

                    # Add a dot only when enough frames have passed (not continuous)
                    frames_since_last_dot += 1
                    if frames_since_last_dot >= 6:  # Slower plotting speed
                        # Prevent intersections by checking if the position is already occupied
                        if (dot_position_x, dot_position_y) not in drawn_points:
                            cv2.circle(canvas, (dot_position_x, dot_position_y), 5, (0, 0, 255), -1)
                            drawn_points.add((dot_position_x, dot_position_y))  # Store the point

                            # Move position based on direction
                            if direction == 0:  # Right
                                dot_position_x += dot_spacing
                                if dot_position_x >= screen_width - 50:
                                    direction = (direction + 1) % 4  # Turn down
                            elif direction == 1:  # Down
                                dot_position_y += dot_spacing
                                if dot_position_y >= screen_height - 50:
                                    direction = (direction + 1) % 4  # Turn left
                            elif direction == 2:  # Left
                                dot_position_x -= dot_spacing
                                if dot_position_x <= 50:
                                    direction = (direction + 1) % 4  # Turn up
                            elif direction == 3:  # Up
                                dot_position_y -= dot_spacing
                                if dot_position_y <= 50:
                                    direction = (direction + 1) % 4  # Turn right

                        frames_since_last_dot = 0  # Reset counter after drawing a dot

                # Display movement status
                cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the main video feed
    cv2.imshow("Camera Movement Detection", frame)

    # Show the dot plotting canvas in full screen
    cv2.namedWindow("Static Camera Plot", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("Static Camera Plot", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Static Camera Plot", canvas)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


Reconnected to the camera.


2025-02-08 20:43:06.016 python[13299:579998] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-08 20:43:06.016 python[13299:579998] +[IMKInputSession subclass]: chose IMKInputSession_Modern
[mjpeg @ 0x116772c50] overread 8
[mjpeg @ 0x116772c50] overread 8


In [13]:
import cv2
import numpy as np
import time
import os

# Replace with your phone's IP address and port
url = "http://192.168.185.1:8080/video"  # Ensure HTTP, not HTTPS

# Function to handle reconnection
def reconnect_camera():
    while True:
        cap = cv2.VideoCapture(url)
        if cap.isOpened():
            print("Reconnected to the camera.")
            return cap
        print("Reconnecting to the camera...")
        cv2.waitKey(1000)  # Wait 1 sec before retrying

# Initialize video capture
cap = reconnect_camera()

# Initialize ORB detector
orb = cv2.ORB_create()

# Read the first frame and detect keypoints
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_keypoints, prev_descriptors = orb.detectAndCompute(prev_gray, None)

# BFMatcher for feature matching
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Get screen resolution for full-screen canvas
screen_width = 1920  # Adjust based on display resolution
screen_height = 1080
canvas = np.ones((screen_height, screen_width, 3), dtype=np.uint8) * 255  # Full-screen white canvas

dot_position_x, dot_position_y = 50, 50  # Start at the **top-left corner**
dot_spacing = 20  # Slower speed by increasing spacing
direction = 0  # 0 = Right, 1 = Down, 2 = Left, 3 = Up
frames_since_last_dot = 0  # Slows down dot plotting

# Timer to skip first 10 seconds
start_time = time.time()
skip_time = 10  # Seconds to skip movement detection
last_rotation_time = 0  # Timer for 5s delay between rotations
rotation_delay = 8  # 5 seconds delay before next rotation

# Store drawn points to prevent intersections
drawn_points = set()

# Directory to save frames
frame_save_path = "saved_frames"
os.makedirs(frame_save_path, exist_ok=True)  # Create directory for saved frames

while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Camera disconnected. Attempting to reconnect...")
        cap = reconnect_camera()
        continue  # Skip frame processing until reconnection

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect keypoints and descriptors in the new frame
    keypoints, descriptors = orb.detectAndCompute(gray, None)

    # Skip movement detection for the first 10 seconds
    elapsed_time = time.time() - start_time
    if elapsed_time < skip_time:
        cv2.putText(frame, "Initializing...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    else:
        if descriptors is not None and prev_descriptors is not None:
            # Match descriptors
            matches = bf.match(prev_descriptors, descriptors)
            matches = sorted(matches, key=lambda x: x.distance)  # Sort by best match

            # Calculate average displacement of keypoints
            motion_x = []
            motion_y = []
            
            for m in matches:
                pt1 = prev_keypoints[m.queryIdx].pt  # Previous keypoint
                pt2 = keypoints[m.trainIdx].pt  # Current keypoint

                motion_x.append(pt2[0] - pt1[0])  # X movement
                motion_y.append(pt2[1] - pt1[1])  # Y movement

            if motion_x and motion_y:
                avg_x = np.mean(motion_x)  # Average movement in X
                avg_y = np.mean(motion_y)  # Average movement in Y
                total_movement = np.sqrt(avg_x**2 + avg_y**2)

                # Set threshold for significant movement
                if total_movement > 200:  
                    movement_text = "Camera Moving"
                    color = (0, 0, 255)  # Red

                    # Rotate direction only if delay period has passed
                    current_time = time.time()
                    if current_time - last_rotation_time >= rotation_delay:
                        direction = (direction + 1) % 4  # Rotate direction
                        last_rotation_time = current_time  # Update last rotation time

                else:
                    movement_text = "Camera Static"
                    color = (0, 255, 0)  # Green

                    # Add a dot only when enough frames have passed (slower plotting)
                    frames_since_last_dot += 1
                    if frames_since_last_dot >= 6:  # Slower by increasing frames delay
                        # Prevent intersections by checking if the position is already occupied
                        if (dot_position_x, dot_position_y) not in drawn_points:
                            cv2.circle(canvas, (dot_position_x, dot_position_y), 5, (0, 0, 255), -1)
                            drawn_points.add((dot_position_x, dot_position_y))  # Store the point

                            # Save frame with filename as coordinates
                            frame_name = f"{dot_position_x}_{dot_position_y}.jpeg"
                            frame_path = os.path.join(frame_save_path, frame_name)
                            cv2.imwrite(frame_path, frame)  # Save the frame

                            # Move position based on direction
                            if direction == 0:  # Right
                                dot_position_x += dot_spacing
                                if dot_position_x >= screen_width - 50:
                                    direction = (direction + 1) % 4  # Turn down
                            elif direction == 1:  # Down
                                dot_position_y += dot_spacing
                                if dot_position_y >= screen_height - 50:
                                    direction = (direction + 1) % 4  # Turn left
                            elif direction == 2:  # Left
                                dot_position_x -= dot_spacing
                                if dot_position_x <= 50:
                                    direction = (direction + 1) % 4  # Turn up
                            elif direction == 3:  # Up
                                dot_position_y -= dot_spacing
                                if dot_position_y <= 50:
                                    direction = (direction + 1) % 4  # Turn right

                        frames_since_last_dot = 0  # Reset counter after drawing a dot

                # Display movement status
                cv2.putText(frame, movement_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

    # Show the main video feed
    cv2.imshow("Camera Movement Detection", frame)

    # Show the dot plotting canvas in fullscreen mode
    cv2.namedWindow("Static Camera Plot", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("Static Camera Plot", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Static Camera Plot", canvas)

    # Update previous frame data
    prev_gray = gray.copy()
    prev_keypoints, prev_descriptors = keypoints, descriptors

    # Press 'q' to exit the video
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Save final canvas as an image
final_frame_path = os.path.join(frame_save_path, "final_frame.jpeg")
cv2.imwrite(final_frame_path, canvas)
print(f"Final frame saved as {final_frame_path}")

# Release resources
cap.release()
cv2.destroyAllWindows()


Reconnected to the camera.


[mjpeg @ 0x3047ffc90] overread 8


Final frame saved as saved_frames/final_frame.jpeg


In [ ]:
# crack detection YOLO
#%pip install torch opencv-python numpy torchvision pillow 


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#%.\crack_detection\Scripts\activate
#%pip install ultralytics


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import torch
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image
from torch import nn
import torch
import torch.nn.functional as F
from ultralytics import YOLO
from collections import deque

In [15]:
#model_path = "best.pt" 
#model = YOLO(model_path).to("cpu")

#def yolov8(image_path):
    #image_path = "saved_frames/370_250.jpeg"  # Replace with your image path
    #frame = cv2.imread(image_path)
    
    # Run YOLOv8 inference on the frame
    #results = model.predict(frame)  # You can also pass a path to a frame image if you need
    #print(len(results[0].boxes.cls))
    #print(results[0].names)
    # Process results (boxes, labels, and confidence scores)
    #boxes = results[0].boxes  # Bounding box data for detected objects
    #for box in boxes:
    #    # Draw bounding boxes around detected cracks
    #    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get the coordinates (x1, y1, x2, y2) of the box
    #    conf = box.conf[0]  # Get the confidence of the detection
        
        # Draw a rectangle and label (optional)
    #    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
    #    cv2.putText(frame, f'{conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the resulting frame with detected cracks
    # cv2.imshow("Crack Detection", frame)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
#yolov8("saved_frames/70_390.jpeg")
#yolov8("saved_frames/50_610.jpeg")

In [ ]:
import cv2
import os
import numpy as np

# Directory containing saved frames
frame_save_path = "saved_frames"
final_frame_path = os.path.join(frame_save_path, "final_frame.jpeg")

# Check if final frame exists
if not os.path.exists(final_frame_path):
    print("Error: final_frame.jpeg not found.")
    exit()

# Load the final frame
final_frame = cv2.imread(final_frame_path)

# Read all filenames from saved_frames directory
dot_filenames = [f for f in os.listdir(frame_save_path) if f.endswith(".jpeg") and f != "final_frame.jpeg"]

model_path = "best.pt" 
model = YOLO(model_path).to("cpu")

# Define green color
green_color = (0, 255, 0)  # Green

# Draw "X" at each dot's saved coordinate in green
for filename in dot_filenames:
    try:
        file_path = frame_save_path+"/"+filename
        frame = cv2.imread(file_path)
        results = model.predict(frame)  # You can also pass a path to a frame image if you need
        c = len(results[0].boxes.cls)
        if c >0:

            # Extract coordinates from filename (format: "X_Y.jpeg")
            coords = filename.replace(".jpeg", "").split("_")
            x, y = int(coords[0]), int(coords[1])
            if x<370:
                # Draw "X" at the coordinates in green
                cv2.line(final_frame, (x - 5, y - 5), (x + 5, y + 5), green_color, 2)
                cv2.line(final_frame, (x - 5, y + 5), (x + 5, y - 5), green_color, 2)
            else:
                pass
        else:
            pass
    except Exception as e:
        print(f"Error processing {filename}: {e}")

# Save and display the updated final frame
updated_final_frame_path = os.path.join(frame_save_path, "final_frame_with_x.jpeg")
cv2.imwrite(updated_final_frame_path, final_frame)
#print(f"Updated final frame saved as {updated_final_frame_path}")

# Show the result
cv2.imshow("Final Frame with Green X Marks", final_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 2 cracks, 67.5ms
Speed: 2.1ms preprocess, 67.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cracks, 59.9ms
Speed: 1.9ms preprocess, 59.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 59.8ms
Speed: 1.7ms preprocess, 59.8ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.2ms
Speed: 1.7ms preprocess, 61.2ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 61.9ms
Speed: 1.7ms preprocess, 61.9ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.7ms
Speed: 1.6ms preprocess, 60.7ms inference, 0.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cracks, 63.1ms
Speed: 1.6ms preprocess, 63.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cracks, 62.8ms
Speed: 1.5ms preprocess, 62.8ms inference, 2.9ms postprocess 

: 